# Improved Media Content Search - Interactive Application

Importing and loading dependencies, please wait...

Note: `%%capture` jupyter flag is used to suppress output. If model fails to load, edit app and re-enable output.

In [4]:
%%capture
!pip3 install tensorflow
!pip3 install tensorflow_hub
!pip3 install annoy
!pip3 install pickle5

import logging
import numpy as np
import pickle5 as pickle

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
tf.get_logger().setLevel(logging.ERROR)
import tensorflow_hub as hub
from annoy import AnnoyIndex

from IPython.display import clear_output

from sklearn import random_projection

In [5]:
# Retrieves the embedding for a batch of sentences 
# Still needed to embed query data

vector_length = 512
index_file_name = "annoy_index_512_20_trees"

embed_module = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/1")
placeholder = tf.placeholder(dtype=tf.string)
embed = embed_module(placeholder)
session = tf.Session()
session.run([tf.global_variables_initializer(), tf.tables_initializer()])
transformer = random_projection.GaussianRandomProjection(vector_length)

def get_embeddings(sentences):
    """Gets embeddings for a given line (document)
    """
    
    print("Getting embeddings...")
    embeddings = session.run(embed, feed_dict={placeholder: sentences})
    
    ## Reduction of dimensionality does not work - must use 512 vector length
    # print("Reducing dimensionality...")
    # reduced_embeddings = transformer.fit_transform(embeddings)
    
    return embeddings

# 1. Load the Model 

Loading the saved model...

In [5]:
ann = AnnoyIndex(vector_length, metric='angular')
ann.load(index_file_name, prefault=True)
print('annoy index loaded.')
with open(index_file_name + '.mapping', 'rb') as handle:
    mapping = pickle.load(handle)
print('mapping file loaded.')

annoy index loaded.
mapping file loaded.


# 2. Interactive Application

In [6]:
%%capture
!pip install ipywidgets

In [7]:
from IPython.display import display, clear_output
import ipywidgets as widgets

# Input for query and number of results
query = widgets.Text(
        value='Sitting in my truck',
        description='Input query:')

slider = widgets.IntSlider(
         value=5,
         min=0,
         max=20,
         step=1,
         description='# of results:')      

button = widgets.Button(description='Submit')
interact = widgets.TwoByTwoLayout(top_left=query, top_right=button,
                       bottom_left=slider)

display(interact)

out = widgets.Output()
display(out)

# define the action, should clear results and leave widget
def on_button_click(b):
    with out:
        clear_output(wait=True)
        user_query = query.value
        print("running query...")

        query_embeddings = get_embeddings([user_query])[0]
        nns = ann.get_nns_by_vector(query_embeddings, slider.value, include_distances=False)

        print("Top {} results for \'{}\'\n".format(slider.value, query.value))
        for idx, item in enumerate(nns):
            print("{}. {} - {}:".format(idx+1, mapping[item][1], mapping[item][2]))
            for x in range(item-3, item+3):
                if x == item:
                    print("==== {} ====".format(mapping[x][0]))
                else:
                    print("     {}     ".format(mapping[x][0]))

            print("\n")

button.on_click(on_button_click)


TwoByTwoLayout(children=(Text(value='Sitting in my truck', description='Input query:', layout=Layout(grid_area…

Output()